In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# installs
! pip install -U spacy
! pip install spacy-universal-sentence-encoder

     |████████████████████████████████| 6.0 MB 9.8 MB/s 
     |████████████████████████████████| 653 kB 55.1 MB/s 
     |████████████████████████████████| 10.1 MB 36.4 MB/s 
     |████████████████████████████████| 181 kB 44.9 MB/s 
     |████████████████████████████████| 42 kB 896 kB/s 
     |████████████████████████████████| 451 kB 38.0 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
     |████████████████████████████████| 462 kB

In [17]:
import numpy as np
import pandas as pd
import seaborn as sns 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, jaccard_score
# imports
import spacy 
from spacy import cli
import spacy_universal_sentence_encoder

In [4]:
train = pd.read_csv("/content/drive/MyDrive/Capstone B3/Phase 3/datasets/merged_df.csv") 
train.sample(3)

,Quesh,Quesh Date,Avg Rating,Total Respo,Type,sent_ans,sent_quesh,charcount
74,Would you rather move every year or live in th...,2021-12-08,3.63,24,2,0.875000,0.0,68.166667
144,What is your favorite morning beverage?,2021-09-29,4.88,8,2,0.125000,2.0,51.375000
96,What are three must-haves in your dream home?,2021-11-16,4.33,15,2,2.066667,1.0,89.866667


In [5]:
test = pd.read_csv("/content/drive/MyDrive/Capstone B3/Phase 3/datasets/100 Quesh - Team B3.csv") 

In [6]:
# download the normal model
model = "en_core_web_md"
cli.download(model)
nlp = spacy.load(model)

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [7]:
# add the pipe
nlp.add_pipe('universal_sentence_encoder', config={'enable_cache': False, 'model_name':'en_use_md'})

Downloaded https://tfhub.dev/google/universal-sentence-encoder/4, Total size: 987.47MB



In [8]:
nlp.pipe_names

['tok2vec',
 'tagger',
 'parser',
 'attribute_ruler',
 'lemmatizer',
 'ner',
 'universal_sentence_encoder']

In [9]:
# make sure we have the old spacy model
nlp_old = spacy.load(model)
nlp_old.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [10]:
# create the 3 representations

# cv
cv = CountVectorizer()
cv.fit(train.Quesh)
dtm = cv.transform(train.Quesh)
print(dtm.shape)

# spacy vectors
docs = list(nlp_old.pipe(train.Quesh, disable=['tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']))
dvs = [doc.vector for doc in docs]
dvs = np.array(dvs)
print(dvs.shape)

# the use embeddings
uses = list(nlp.pipe(train.Quesh, disable=['tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']))
uvs = [doc.vector for doc in uses]
uvs = np.array(uvs)
print(uvs.shape)


(155, 406)
(155, 300)
(155, 512)


In [11]:
# fit the models

# count vects
knn = KNeighborsClassifier(3)
knn.fit(dtm.toarray(), train['Type'])
preds_cv = knn.predict(dtm.toarray())

# spacy pretrained
knn = KNeighborsClassifier(3)
knn.fit(dvs, train['Type'])
preds_spacy = knn.predict(dvs)

# universal sentence encodings
knn = KNeighborsClassifier(3)
knn.fit(uvs, train['Type'])
preds_use = knn.predict(uvs)

In [12]:
# put everything back
train[['cv', 'spacy', 'use']] = np.stack([preds_cv, preds_spacy, preds_use], axis=1)

In [13]:
train.head(3)

,Quesh,Quesh Date,Avg Rating,Total Respo,Type,sent_ans,sent_quesh,charcount,cv,spacy,use
0,What is something that you've learned to accept?,2022-02-20,3.78,23,2,1.565217,1.0,113.869565,2,2,2
1,What was your favorite food growing up and wha...,2022-02-19,4.19,16,2,2.750000,3.0,93.437500,2,2,2
2,What form of art fascinates you most?,2022-02-18,4.26,23,2,3.043478,3.0,102.000000,2,2,2


In [14]:
# accuracy
print(accuracy_score(train.Type, train.cv))
print(accuracy_score(train.Type, train.spacy))
print(accuracy_score(train.Type, train.use))

0.9290322580645162
0.9161290322580645
0.8193548387096774


In [21]:
from sklearn.metrics import classification_report

In [23]:
print(classification_report(train.Type, train.cv))

              precision    recall  f1-score   support

           1       0.83      0.91      0.87        11
           2       0.94      0.97      0.95       119
           3       1.00      0.65      0.79        17
           4       0.88      0.88      0.88         8

    accuracy                           0.93       155
   macro avg       0.91      0.85      0.87       155
weighted avg       0.93      0.93      0.93       155

